In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
#getting the dataset using pytorch
from torchvision import datasets, transforms  

# Download MNIST dataset  
mnist_train = datasets.MNIST(root="./data", train=True, download=True)#transform=transforms.ToTensor())  
mnist_test = datasets.MNIST(root="./data", train=False, download=True)#, transform=transforms.ToTensor())  

# Check dataset length  
print(len(mnist_train), len(mnist_test)) 

In [3]:
from tensorflow.keras.datasets import mnist  

# Load MNIST dataset  
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

11501568/11490434 [==============================] - 8s 1us/step


In [11]:
x_train.flatten().shape

(47040000,)

In [15]:
x_train[0].shape

(28, 28)

In [25]:
x_train = x_train.reshape((60000, 784)).transpose()
x_test = x_test.reshape((10000, 784)).transpose()

In [27]:
#Now each column is a 28x28 image
x_train[:,0].shape

(784,)

In [ ]:
def initialzing_params():
    W1 = np.random.rand(10, 784) - 0.25
    b1 = np.random.rand(10, 784) - 0.25
    
    W2 = np.random.rand(10, 10) - 0.25
    b2 = np.random.rand(10, 10) - 0.25
    
    return W1, b1, W2, b2

def f_prop(W1, b1, W2, b2, X):